In [111]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Embedding, LSTM, Dense 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from gensim.utils import simple_preprocess
import numpy as np 
import regex as re 
import gzip

In [112]:

def read(file_path): 
    with open(file_path, 'r',encoding='utf-8') as file: 
        text = file.read() 
    sentences = [sentence.strip() for sentence in re.split( 
        r'(?<=[.!?"])\s+', text) if sentence.strip()] 
  
    return sentences 
  
file_path = 'story1.txt'
text_data = read(file_path)[:1000]
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(text_data) 
total_words = len(tokenizer.word_index) + 1
input_sequences = [] 
for line in text_data: 
    token_list = tokenizer.texts_to_sequences([line])[0] 
    for i in range(1, len(token_list)): 
        n_gram_sequence = token_list[:i+1] 
        input_sequences.append(n_gram_sequence) 

seq_size = max([len(seq) for seq in input_sequences]) 
input_sequences = np.array(pad_sequences( 
    input_sequences, maxlen=seq_size, padding='pre')) 
X, y = input_sequences[:, :-1], input_sequences[:, -1] 
y=tf.keras.utils.to_categorical(y,num_classes=total_words)


In [113]:
model=Sequential()
model.add(Embedding(total_words,10,input_length=seq_size-1))
model.add(LSTM(128))
model.add(Dense(total_words,activation='softmax'))
optimizer=tf.kers.optimizers.Adam()
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [120]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)
x_test.size

713277

In [121]:
model.fit(x_train,y_train,epochs=100,verbose=1)

Epoch 1/100
423/423 [==============================] - 34s 81ms/step - loss: 6.2299 - accuracy: 0.0616
Epoch 2/100
423/423 [==============================] - 41s 97ms/step - loss: 5.9773 - accuracy: 0.0659
Epoch 3/100
423/423 [==============================] - 35s 82ms/step - loss: 5.7833 - accuracy: 0.0701
Epoch 4/100
423/423 [==============================] - 35s 83ms/step - loss: 5.6017 - accuracy: 0.0751
Epoch 5/100
423/423 [==============================] - 36s 86ms/step - loss: 5.4200 - accuracy: 0.0797
Epoch 6/100
423/423 [==============================] - 35s 82ms/step - loss: 5.2310 - accuracy: 0.0852
Epoch 7/100
423/423 [==============================] - 36s 84ms/step - loss: 5.0412 - accuracy: 0.0939
Epoch 8/100
423/423 [==============================] - 35s 84ms/step - loss: 4.8533 - accuracy: 0.1023
Epoch 9/100
423/423 [==============================] - 36s 86ms/step - loss: 4.6640 - accuracy: 0.1095
Epoch 10/100
423/423 [==============================] - 36s 86ms/step - l

In [139]:
import random
seed_text="i think"
next_words=10
  
for _ in range(next_words): 
    token_list = tokenizer.texts_to_sequences([seed_text])[0] 
    token_list = pad_sequences( [token_list], maxlen=seq_size-1, padding='pre') 
    predicted_probs = model.predict(token_list) 
    predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
    seed_text += " " + predicted_word 
  
print(seed_text) 

1/1 [==============================] - 0s 30ms/step
i think perhaps it is almost time that the door always there
